# Decoding Representations in Deep Networks


Decoding representations in the different layers of a deep network is a very valuable machine learning excercise, but it is not only this.

The general methodology of this task is the same as for decoding representations in networks of real neurons.

Usually one wants to decode a stimulus category or the continuous value of one of its parameters, such as luminosity, eccentricity, position etc.

The steps are :

- train a linear classifier or a linear regression algorithm
- assess its performance

This operation entails hyperparameters search with cross-validation


In [1]:
# imports

import os
import os.path as path
from os import listdir 
from os.path import isfile, join
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import h5py

from keras.datasets import mnist
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical
from keras import backend as K

from tqdm import tqdm

Using TensorFlow backend.


# Load data, model and extract representations

We repeat the steps in the preceding exercise first

In [2]:
def load_mnist_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train/255.0, x_test/255.0
    x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)    
    x_train, x_test = np.expand_dims(x_train, 3), np.expand_dims(x_test, 3)
    y_test_int = y_test # keep labels in integer form
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)
    return x_train, x_test, y_train, y_test, y_test_int

In [3]:
x_train, x_test, y_train, y_test, y_test_int = load_mnist_data()

11493376/11490434 [==============================] - 9s 1us/step


In [8]:
# extract a random sample of n_sample elements from each category in test set
n_samples = 100
samples = np.zeros((n_samples*10,28,28,1))
labels = np.zeros((n_samples*10,))
labels_int = []
for i in range(10):
    labels_int.append([i]*n_samples)
    mask = y_test_int == i
    temp_x_test = x_test[mask]
    idx = np.random.permutation(temp_x_test.shape[0])[:n_samples]
    samples[i*n_samples: (i+1)*n_samples,:,:,:] = temp_x_test[idx]
    labels[i*n_samples: (i+1)*n_samples] = np.zeros((n_samples,)) + i
labels = to_categorical(labels)

In [9]:
labels_int = np.asarray(labels_int).flatten()

In [10]:
model = load_model(join('models', 'mlp3.h5'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
images (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
hidden1 (Dense)              (None, 100)               78500     
_________________________________________________________________
hidden2 (Dense)              (None, 100)               10100     
_________________________________________________________________
hidden3 (Dense)              (None, 100)               10100     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 99,710
Trainable params: 99,710
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
def extract_representations(model,samples):
    inp = model.input                                           # input placeholder
    outputs = [layer.output for layer in model.layers]          # all layer outputs
    functor = K.function([inp, K.learning_phase()], outputs )   # evaluation function
    layers_outs = functor([samples, 1.])
    return layers_outs

In [12]:
layers_outs = extract_representations(model,samples)

In [13]:
layers_names = [l.name for l in model.layers]

In [14]:
layers_names

['images', 'flatten_1', 'hidden1', 'hidden2', 'hidden3', 'output']

In [15]:
# print names
_ = [print(l_name, l_out.shape) for l_name,l_out in zip(layers_names, layers_outs)]

images (1000, 28, 28, 1)
flatten_1 (1000, 784)
hidden1 (1000, 100)
hidden2 (1000, 100)
hidden3 (1000, 100)
output (1000, 10)


In [16]:
ind_to_extract = [0,2,3,4,5]
layers_labels = ['images', 'hidden1', 'hidden2', 'hidden3', 'output']

# Decoding of Representations in a Layer

In [17]:
# flatten representations
layers_outs = [np.reshape(l, (n_samples*10, -1)) for l in layers_outs]

# print names
_ = [print(l_name, l_out.shape) for l_name,l_out in zip(layers_names, layers_outs)]

images (1000, 784)
flatten_1 (1000, 784)
hidden1 (1000, 100)
hidden2 (1000, 100)
hidden3 (1000, 100)
output (1000, 10)


In [18]:
layers_info = zip(layers_names, layers_outs)

In [19]:
from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import preprocessing

In [20]:
from sklearn.metrics import make_scorer, balanced_accuracy_score
from scipy.stats import pearsonr

def discrete_variable_score(y_true, y_predicted):    
    return balanced_accuracy_score(y_true, y_predicted)

def continuous_variable_score(y_true, y_predicted):
    return pearsonr(y_true,y_predicted)[0]

In [21]:
# create scorer
my_func = make_scorer(discrete_variable_score,greater_is_better=True)

In [22]:
Score = []
Results = []
Parameters = []


Cs = np.power(10., np.arange(-3,3))
#Cs = [0.01]

# define grid of parameters
param_grid = {'C' : Cs}
nresampling = 10
nfolds = 5
nprocs = 1


y = labels_int

for i in tqdm(ind_to_extract):
    
    X = layers_outs[i]
    
    # preprocessing
    #X = preprocessing.scale(X) 
    
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=None)
    
    # basis classifier
    clf_ = SVC(kernel='linear')
    
    grid_search = GridSearchCV(clf_, param_grid, 
                                scoring=my_func, 
                                cv=nfolds, 
                                n_jobs=nprocs, 
                                refit=True,
                                return_train_score=True)

    grid_search.fit(X_train, y_train)    
    
    
    Score.append(grid_search.best_score_)             
    Results.append(grid_search.cv_results_)
    Parameters.append(grid_search.best_params_['C'])

  0%|          | 0/5 [00:00<?, ?it/s]/home/qsb/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
 20%|██        | 1/5 [00:28<01:53, 28.43s/it]/home/qsb/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
100%|██████████| 5/5 [00:38<00:00,  8.54s/it]


In [23]:
fs = 20
ms = 20

fig = plt.figure(figsize=(10,10))
plt.plot(range(len(ind_to_extract) ), Score, '-bo',markersize=ms) 
plt.xticks(range(len(ind_to_extract) ), 
           layers_labels, 
           rotation='vertical',
           fontsize=fs)

plt.yticks(fontsize=fs)
plt.ylabel('balanced accuracy',fontsize=fs)
plt.xlabel('layer',fontsize=fs)
plt.show()